In [412]:
import re
import pandas as pd

## Q1

In [413]:
def q1(l):
    #Func calcalate sum list with filter
    #param: number int
    #param: l  list
    #return list 
    if not isinstance(l, list):
        raise Exception
        
    result = list()      
    for number, _ in enumerate(l):
        result.append(sum([elem for n, elem in enumerate(l) if n!=number]))
        
    return result

# Q2

In [414]:
data  = pd.read_csv('data job posts.csv')
data.head()

,jobpost,date,Title,Company,AnnouncementCode,Term,Eligibility,Audience,StartDate,Duration,...,Salary,ApplicationP,OpeningDate,Deadline,Notes,AboutC,Attach,Year,Month,IT
0,AMERIA Investment Consulting Company\r\nJOB TI...,"Jan 5, 2004",Chief Financial Officer,AMERIA Investment Consulting Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"To apply for this position, please submit a\r\...",NaN,26 January 2004,NaN,NaN,NaN,2004,1,False
1,International Research & Exchanges Board (IREX...,"Jan 7, 2004",Full-time Community Connections Intern (paid i...,International Research & Exchanges Board (IREX),NaN,NaN,NaN,NaN,NaN,3 months,...,NaN,Please submit a cover letter and resume to:\r\...,NaN,12 January 2004,NaN,The International Research & Exchanges Board (...,NaN,2004,1,False
2,Caucasus Environmental NGO Network (CENN)\r\nJ...,"Jan 7, 2004",Country Coordinator,Caucasus Environmental NGO Network (CENN),NaN,NaN,NaN,NaN,NaN,Renewable annual contract\r\nPOSITION,...,NaN,Please send resume or CV toursula.kazarian@......,NaN,20 January 2004\r\nSTART DATE: February 2004,NaN,The Caucasus Environmental NGO Network is a\r\...,NaN,2004,1,False
3,Manoff Group\r\nJOB TITLE: BCC Specialist\r\n...,"Jan 7, 2004",BCC Specialist,Manoff Group,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Please send cover letter and resume to Amy\r\n...,NaN,23 January 2004\r\nSTART DATE: Immediate,NaN,NaN,NaN,2004,1,False
4,Yerevan Brandy Company\r\nJOB TITLE: Software...,"Jan 10, 2004",Software Developer,Yerevan Brandy Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Successful candidates should submit\r\n- CV; \...,NaN,"20 January 2004, 18:00",NaN,NaN,NaN,2004,1,True


In [415]:
def parseJobPost(text, key=None):
    """
    parsing cell postjob
    :param text
    :key: text
    :return:  dict
    """
    d_output = dict()
    columns = ['Job Title', 'Position Duration', 
               'Position Location', 'Job Description',
               'Job Responsibilities','Required Qualifications',
               'Remuneration', 'Application Deadline','About Company'] 
    for i in columns:
        res = re.findall(r'{}:([^:]+)'.format(i), text, re.IGNORECASE)
        d_output[i] = ' '.join(res[0].split('\r\n')[:-1]).strip() if res else None
    
    return d_output if not key else d_output[key]

In [416]:
## Identify the company with the most number of job ads in the past 2 years
YEAR = 2004
data[data['Year'] > YEAR].Company.value_counts()

ArmenTel CJSC                                                                                        353
Mentor Graphics Development Services CJSC                                                            236
Career Center NGO                                                                                    228
World Vision Armenia                                                                                 206
Orange Armenia                                                                                       203
Ameriabank CJSC                                                                                      196
Converse Bank CJSC                                                                                   161
SAS Group LLC                                                                                        150
UNDP Armenia Office                                                                                  132
Energize Global Services CJSC                          

In [417]:
##  Identify the month with the largest number of job ads over the years
months = ['Jan', 'Feb', 'Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov', 'Dec']
m = data.Month.value_counts()
for key,value in m.items():
    print('{}\t {}'.format(months[key-1], value))

Mar	 1702
Feb	 1665
Jun	 1662
Sep	 1652
Oct	 1637
Aug	 1613
Jul	 1595
May	 1580
Nov	 1573
Apr	 1466
Dec	 1432
Jan	 1424


In [418]:
##  Clean text and generate new text from Job Responsibilities column: The new text
## shall not contain any stop words, and the plural words shall be converted into
## singular words.
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
# if show error uncomment and download packages
# import nltk
# nltk.download()

# import nltk
# nltk.download('punkt')
# nltk.download('wordnet')

stop_words = set(stopwords.words('english'))

def filterStopWords(text):
    """
    rm stop words from text
     param: text
     return text
     """
    word_tokens = word_tokenize(text)
    return ' '.join([w for w in word_tokens if not w in stop_words])

def toSingular(text):
    """ convert from plural to singular
     param text
     return text
     """
    wnl = WordNetLemmatizer()
    word_tokens = word_tokenize(text)    
    tokens = [token.lower() for token in word_tokens]
    return (' ').join([wnl.lemmatize(token) for token in tokens])

def setTitle(sentences):
    """ First letter set title in sentence 
     param: list
     return str
    """
    result = list()
    for sentence in sentences:
        sentence_new = list()
        for n, w in enumerate(sentence.split(' ')):
            word = w.title() if n == 0 else w
            sentence_new.append(word)
            
        result.append((' ').join(sentence_new))
        
    return '.'.join(result)
 
def doText(jobpost, key='Job Responsibilities'):
    """
    new text from jobpost.
    :param text
    :return text
    """
    text = parseJobPost(jobpost, key)        
    if not text:    
        return None
    
    text_without_stop = filterStopWords(text)
    text_singlular = toSingular(text_without_stop)    
    list_sentence = [ sentence.strip() for sentence in text_singlular.split('.')] #
    
    return setTitle(list_sentence)
## rm and set new responsibles
data['JobRequirment'] = data['jobpost'].apply(lambda x: doText(x))
data['JobDescription'] = data['jobpost'].apply(lambda x: doText(x, 'Job Description'))

In [419]:
## Write functions to identify null/NA values and to replace null/NA values with a
## custom message in “Duration” column
CUSTOM = 0
data['Duration'] = data['Duration'].fillna(CUSTOM) 

In [420]:
import sqlite3
conn = sqlite3.connect('base_data_job_posts.sqlite.db')
data.to_sql('base', conn)
conn.close()

## Q3

In [421]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

In [422]:
def doClasters(key, n_clusters=10, top_words=10):    
    """
    Clasters
    :param key:str
    :n_clasters: int
    :top_words : int
    :return: list
    """
    l_result = list()
    #drop all NaN
    data_without_nan = data[data[key].notnull()]
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform(data_without_nan)
    kmeans = KMeans(n_clusters=n_clusters, init='k-means++',max_iter=100, n_init=1)
    kmeans.fit(X)
    order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names()
    for i in range(n_clusters):
        tmp = list()
        for ind in order_centroids[i, :top_words]:
            tmp.append(terms[ind])
            
        l_result.append(tmp)
        
    return l_result

In [425]:
#list(data)
top_words = doClasters('RequiredQual',n_clusters=5, top_words=10)
with open('top_words.txt', 'w') as f:
    for line in top_words:
        f.write(' '.join(line))